# A new test.
Making sure that I can loop through the different people.

In [5]:
import requests
from bs4 import BeautifulSoup

ssl_verify_path = 'ch.pem'
#idn = 13141
for idn in range(1,3):
    print("IDN: ", idn)
    url = f'https://genealogy.math.ndsu.nodak.edu/id.php?id={idn}'
    print("URL: ",url)
    page = requests.get(url,verify=ssl_verify_path)


    soup = BeautifulSoup(page.content,'html.parser')

    # Get all of the fields I want, and print so it is neat
    name = soup.find('h2')
    name_string = name.text.strip()

    mathsci = name.find_next('p')
    mathsci_link = name.find_next('a')
    mathsci_string = mathsci_link['href']

    # The spans are nested, so pull the full string which gives [phd, university, year]
    # Split list into two to combine everything but year, then get year as last element
    degree_year = mathsci.find_next('span')
    degree_year_string = degree_year.text.strip().rsplit(None,1)[1] # probably want as date for db

    college = degree_year.find_next('span')
    college_string = college.text.strip()

    dissertation = college.find_next('div')
    dissertation_string = dissertation.text.strip().split('\n')
    dissertation_string_final = dissertation_string[len(dissertation_string)-1]

    classification = dissertation.find_next('div')
    classification_string = classification.text.strip()
    classification_split_text = classification_string.split(':')[1]


    # It is combined on an em dash
    # Need to regex to remove
    import re
    classification_split_utf = re.sub(u'\u2014','--',classification_split_text)

    classification_number = str(classification_split_utf.split('--')[0]).strip()
    classification_name_string = classification_split_utf.split('--')[1]

    academic = {
        "Name": name_string,
        "MathSci": mathsci_string,
        "Degree Year": degree_year_string,
        "University": college_string,
        "Dissertation Title": dissertation_string_final,
        "Math Genre Number": classification_number,
        "Math Subfield": classification_name_string
    }

    for attribute in academic:
        print(academic[attribute])
    print("--------------------------\n")

IDN:  1
URL:  https://genealogy.math.ndsu.nodak.edu/id.php?id=1
Ernest Willard Anderson
id.php?id=258
1933
Iowa State University
Statics of Special Types of Homogeneous Elastic Slabs with Variable Thickness
74
Mechanics of deformable solids
--------------------------

IDN:  2
URL:  https://genealogy.math.ndsu.nodak.edu/id.php?id=2
Archie  Higdon
id.php?id=258
1936
Iowa State University
Stresses in Moderately Thick Rectangular Plates
74
Mechanics of deformable solids
--------------------------



# Issue
OK, so it is working. The issue is that I am looking for the next href, which only exists if the person has a MathSci account. So, I can just put an if statement to check for that. 

In [10]:
import requests
from bs4 import BeautifulSoup

ssl_verify_path = 'ch.pem'
#idn = 13141
for idn in range(790,800):
    url = f'https://genealogy.math.ndsu.nodak.edu/id.php?id={idn}'
    page = requests.get(url,verify=ssl_verify_path)


    soup = BeautifulSoup(page.content,'html.parser')

    # Get all of the fields I want, and print so it is neat
    name = soup.find('h2')
    name_string = name.text.strip()

    mathsci = name.find_next('p')
    mathsci_string = mathsci.text.strip()
    if mathsci_string == 'MathSciNet':
        mathsci_link = name.find_next('a')
        mathsci_string = mathsci_link['href']
    else:
        mathsci_string = ''

    # The spans are nested, so pull the full string which gives [phd, university, year]
    # Split list into two to combine everything but year, then get year as last element
    degree_year = mathsci.find_next('span')
    degree_year_string = degree_year.text.strip().rsplit(None,1)[1] # probably want as date for db

    college = degree_year.find_next('span')
    college_string = college.text.strip()

    dissertation = college.find_next('div')
    dissertation_string = dissertation.text.strip().split('\n')
    dissertation_string_final = dissertation_string[len(dissertation_string)-1]

    classification = dissertation.find_next('div')
    classification_string = classification.text.strip()
    classification_split_text = classification_string.split(':')[1]


    # It is combined on an em dash
    # Need to regex to remove
    import re
    classification_split_utf = re.sub(u'\u2014','--',classification_split_text)

    classification_number = str(classification_split_utf.split('--')[0]).strip()
    classification_name_string = classification_split_utf.split('--')[1]

    academic = {
        "Name": name_string,
        "MathSci": mathsci_string,
        "Degree Year": degree_year_string,
        "University": college_string,
        "Dissertation Title": dissertation_string_final,
        "Math Genre Number": classification_number,
        "Math Subfield": classification_name_string
    }

    for attribute in academic:
        print(academic[attribute])
    print("--------------------------")

IndexError: list index out of range

Same Issue here. Need to check if it exists first.

In [22]:
import requests
from bs4 import BeautifulSoup


ssl_verify_path = 'ch.pem'
#idn = 13141
for idn in range(267785,267786):
    url = f'https://genealogy.math.ndsu.nodak.edu/id.php?id={idn}'
    page = requests.get(url,verify=ssl_verify_path)


    soup = BeautifulSoup(page.content,'html.parser')

    # Get all of the fields I want, and print so it is neat
    name = soup.find('h2')
    name_string = name.text.strip()

    mathsci = name.find_next('p')
    mathsci_string = mathsci.text.strip()
    if mathsci_string == 'MathSciNet':
        mathsci_link = name.find_next('a')
        mathsci_string = mathsci_link['href']
    else:
        mathsci_string = ''

    # The spans are nested, so pull the full string which gives [phd, university, year]
    # Split list into two to combine everything but year, then get year as last element
    degree_year = mathsci.find_next('span')
    degree_year_string = degree_year.text.strip().rsplit(None,1)[1] # probably want as date for db

    college = degree_year.find_next('span')
    college_string = college.text.strip()

    dissertation = college.find_next('div')
    dissertation_string = dissertation.text.strip().split('\n')
    dissertation_string_final = dissertation_string[len(dissertation_string)-1]

    classification = dissertation.find_next('div')
    classification_string = classification.text.strip()
    classification_number = ''
    classification_name_string = ''
    if classification_string.split(':')[0] == 'Mathematics Subject Classification':
        classification_split_text = classification_string.split(':')[1]

        # It is combined on an em dash
        # Need to regex to remove
        import re
        # Removing with regex
        classification_split_utf = re.sub(u'\u2014','--',classification_split_text)

        classification_number = str(classification_split_utf.split('--')[0]).strip()
        classification_name_string = classification_split_utf.split('--')[1]

    academic = {
        "Name": name_string,
        "MathSci": mathsci_string,
        "Degree Year": degree_year_string,
        "University": college_string,
        "Dissertation Title": dissertation_string_final,
        "Math Genre Number": classification_number,
        "Math Subfield": classification_name_string
    }

    for attribute in academic:
        print(academic[attribute])
    print("--------------------------")

Sergio  López-Ureña
http://www.ams.org/mathscinet/MRAuthorID/1223209
2019
Universitat de València
Multi-scale mathematical techniques for signal processing
41
Approximations and expansions
--------------------------


Looping works!

I think the only thing I have to do now is make sure I can scrape the homepage for the index number, then set up the mysql and sql stuff